<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Regression Models with Keras</font></h1>


<a id="item31"></a>


## Download and Clean Dataset


In [54]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from numpy import std, mean
import keras
from keras.models import Sequential
from keras.layers import Dense

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>


Let's download the data and read it into a <em>pandas</em> dataframe.


In [22]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 


#### Let's check how many data points we have.


In [23]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.


Let's check the dataset for any missing values.


In [24]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [25]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.


#### Split data into predictors and target


The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.


In [26]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>


Let's do a quick sanity check of the predictors and the target dataframes.


In [27]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [28]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.


Let's save the number of predictors to _n_cols_ since we will need this number when building our network.


In [29]:
n_cols = predictors.shape[1] # number of predictors

# Build Model

In [30]:
model = Sequential()
model.add(Dense(50, activation='relu', input_shape=(n_cols,)))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [40]:
def get_mses(model, target, predictors, iterations=50):
    arr_mse = []
    for i in range(iterations):
        arr_mse += [mean_squared_error(target, model.predict(predictors))]
    return arr_mse

# PART A - Train Model and Get Standard Deviation of 50 Runs

In [ ]:
model.fit(predictors, target, validation_split=0.3, epochs=50)

In [41]:
mses_a = get_mses(model, target, predictors, iterations=50)

In [55]:
print('Mean of the Mean Squared Errors: {}'.format(mean(mses_a)))

Mean of the Mean Squared Errors: 66.07249704809739


In [42]:
print('Standard deviation of 50 MSE values for NON-normalized predictors: {}'.format(std(mses_a)))

Standard deviation of 50 MSE values for NON-normalized predictors: 1.4210854715202004e-14


# Part B - Normalize Data

In [43]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()

In [44]:
model.fit(predictors_norm, target, validation_split=0.3, epochs=50)

Train on 721 samples, validate on 309 samples
Epoch 1/50
721/721 [==============================] - 1s 1ms/step - loss: 1682.6333 - val_loss: 1179.6999
Epoch 2/50
721/721 [==============================] - 1s 837us/step - loss: 1562.8682 - val_loss: 1079.9528
Epoch 3/50
721/721 [==============================] - 1s 856us/step - loss: 1392.1105 - val_loss: 944.1418
Epoch 4/50
721/721 [==============================] - 1s 944us/step - loss: 1159.1309 - val_loss: 774.5600
Epoch 5/50
721/721 [==============================] - 1s 1ms/step - loss: 884.8625 - val_loss: 585.9105
Epoch 6/50
721/721 [==============================] - 1s 967us/step - loss: 621.3970 - val_loss: 414.3967
Epoch 7/50
721/721 [==============================] - 1s 918us/step - loss: 414.7461 - val_loss: 288.8625
Epoch 8/50
721/721 [==============================] - 1s 996us/step - loss: 294.1040 - val_loss: 214.4230
Epoch 9/50
721/721 [==============================] - 1s 950us/step - loss: 238.3149 - val_loss: 174.353

In [45]:
mses_b = get_mses(model, target, predictors_norm, iterations=50)

In [46]:
print('Standard deviation of 50 MSE values for normalized predictors: {}'.format(std(mses_b)))

Standard deviation of 50 MSE values for normalized predictors: 2.842170943040401e-14


In [47]:
std(mses_b) < std(mses_a)

False

# Part C - Increase Number of Epochs

In [48]:
model.fit(predictors_norm, target, validation_split=0.3, epochs=100)

Train on 721 samples, validate on 309 samples
Epoch 1/100
721/721 [==============================] - 1s 915us/step - loss: 109.2600 - val_loss: 102.1344
Epoch 2/100
721/721 [==============================] - 1s 832us/step - loss: 107.2197 - val_loss: 102.2289
Epoch 3/100
721/721 [==============================] - 1s 801us/step - loss: 105.3677 - val_loss: 101.7460
Epoch 4/100
721/721 [==============================] - 1s 942us/step - loss: 103.8019 - val_loss: 101.7641
Epoch 5/100
721/721 [==============================] - 1s 965us/step - loss: 102.2722 - val_loss: 99.6458
Epoch 6/100
721/721 [==============================] - 1s 885us/step - loss: 100.8136 - val_loss: 100.1753
Epoch 7/100
721/721 [==============================] - 1s 964us/step - loss: 99.0711 - val_loss: 98.7700
Epoch 8/100
721/721 [==============================] - 1s 1ms/step - loss: 97.8954 - val_loss: 98.3953
Epoch 9/100
721/721 [==============================] - 1s 1ms/step - loss: 96.1430 - val_loss: 98.4407
Ep

In [49]:
mses_c = get_mses(model, target, predictors_norm, iterations=50)

In [50]:
print('Standard deviation of 50 MSE values for normalized predictors, 100 epochs: {}'.format(std(mses_c)))

Standard deviation of 50 MSE values for normalized predictors, 100 epochs: 7.105427357601002e-15


In [51]:
std(mses_b) < std(mses_c)

False

# Part D - Increase Number of Hidden Layers

In [52]:
model = Sequential()
model.add(Dense(50, activation='relu', input_shape=(n_cols,)))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(predictors_norm, target, validation_split=0.3, epochs=50)
mses_d = get_mses(model, target, predictors_norm, iterations=50)

Train on 721 samples, validate on 309 samples
Epoch 1/50
721/721 [==============================] - 2s 3ms/step - loss: 1703.4617 - val_loss: 1224.5214
Epoch 2/50
721/721 [==============================] - 1s 1ms/step - loss: 1676.9986 - val_loss: 1204.1265
Epoch 3/50
721/721 [==============================] - 1s 1ms/step - loss: 1636.6910 - val_loss: 1163.7307
Epoch 4/50
721/721 [==============================] - 1s 1ms/step - loss: 1549.4060 - val_loss: 1075.6584
Epoch 5/50
721/721 [==============================] - 1s 1ms/step - loss: 1364.2927 - val_loss: 895.8429
Epoch 6/50
721/721 [==============================] - 1s 1ms/step - loss: 1011.9580 - val_loss: 593.2750
Epoch 7/50
721/721 [==============================] - 1s 1ms/step - loss: 533.2250 - val_loss: 285.8287
Epoch 8/50
721/721 [==============================] - 1s 1ms/step - loss: 289.3449 - val_loss: 200.0278
Epoch 9/50
721/721 [==============================] - 1s 1ms/step - loss: 247.3833 - val_loss: 189.2317
Epoch 10

In [56]:
print('Standard deviation for part D: {}'.format(std(mses_d)))

Standard deviation for part D: 2.842170943040401e-14


In [57]:
std(mses_b) < std(mses_d)

False